<a href="https://colab.research.google.com/github/yg4236/Machine-Deep_Learning/blob/main/ML1st_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive
import torch.nn as nn
import torch.optim as optim
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X = np.empty((0, 4), dtype=int)
import torch
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 502, kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 502, kernel_size=3,stride=2, padding=0),
            torch.nn.ReLU())

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 502, kernel_size=3,stride=1, padding=0),
            torch.nn.ReLU())

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(502, 502, kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))


        # 전결합층 
        self.fc = torch.nn.Linear(502, 27, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        global X
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        X = np.append(X,out.detach().numpy())
        # print(out.shape)
        # out = self.layer5(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

class BadukDataset():
    def __init__(self, array, label):
        super().__init__()
        self.array = array
        self.label = label
        # self.transforms = transforms

    def __getitem__(self, index):
        final = self.array[index]
        final_label = self.label[index]
        final_label = torch.tensor(final_label,dtype=torch.long)
        return final, final_label

    def __len__(self):
        return len(self.array)


In [ ]:
def train(trainloader, net, criterion, optimizer):
  running_loss = 0.0
  for i, data in enumerate(trainloader,0):
      inputs, labels = data  # data입력
      optimizer.zero_grad()  # gradient 매개변수를 0으로 만듬
      # 순전파 + 역전파 + 최적화
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      # 통계출력
      running_loss += loss.item()
      if i % 200 == 199:
          print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
          running_loss = 0.0

def test(testloader, net):
  # 전체 데이터 셋에 대한 결과 확인
  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  print('Accuracy of the network on the test images: %d %%' % (100*correct/total))

In [ ]:
if __name__ == '__main__':
    device = 'cuda'
    rating = ['18k', '17k', '16k', '15k', '14k', '13k', '12k', '11k', '10k', 
              '9k', '8k', '7k', '6k', '5k', '4k', '3k', '2k', '1k',
              '1d', '2d', '3d', '4d', '5d', '6d', '7d', '8d', '9d']    
    ################################### traindata 정제 ################################################
    #load train data and label
    train_data_path = "/content/drive/My Drive/기계학습 1등팀/data/train_data.npy" # Your train data file
    train_label_path = "/content/drive/My Drive/기계학습 1등팀/data/train_label.npy" # Your train label file

    train_data = np.load(train_data_path, allow_pickle=True)
    train_label_rawdata = np.load(train_label_path, allow_pickle=True)  

    num_train_data = len(train_data)

    # Get last baduk data of all game
    last_train_data = np.zeros( (num_train_data,502, 19,19) )

    for i in range(num_train_data):
      temp_last_data = train_data[i]
      for j in range(len(temp_last_data)):
        last_train_data[i, j, :] = temp_last_data[j]

    # Prepare label for training SVM (character rating to integer rating)
    train_label = np.zeros(num_train_data)
    for i in range(num_train_data):
      train_label_temp = train_label_rawdata[i]
      train_label_idx = rating.index(train_label_temp)
      train_label[i] = train_label_idx
    
    train_dataset = BadukDataset(last_train_data, train_label)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)

    ################################### testdata 정제 ################################################

    # Get test data
    test_data_path = "/content/drive/My Drive/기계학습 1등팀/data/test_data.npy"
    test_data = np.load(test_data_path, allow_pickle=True)

    # Get last test data
    num_test_data = len(test_data)

    last_test_data = np.zeros( (num_test_data,502, 19, 19) )

    for i in range(num_test_data):
      temp_last_data = test_data[i]
      for j in range(len(temp_last_data)):
        last_test_data[i, j, :] = temp_last_data[j]

    test_label_path = "/content/drive/My Drive/기계학습 1등팀/data/test_label2.npy"
    test_label_rawdata = np.load(test_label_path, allow_pickle=True)
    test_label = np.zeros(num_test_data)

    test_dataset = BadukDataset(last_test_data, test_label)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=0)

    ################################## 학습 하는 부분 #################################################
    net = CNN().double()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print (device,"로 학습")

    # 신경망 학습하기
    for epoch in range(40):
        train(trainloader, net, criterion, optimizer)
        test(testloader, net)
        PATH = '/content/drive/MyDrive/기계학습 1등팀/data/YG/baduk_net1_%d.pth'%epoch
        torch.save(net, PATH) 
    print('Finished Training')
    np.save('/content/drive/MyDrive/기계학습 1등팀/data/Output_train_1.npy', X)

In [ ]:
########################################## testset feature 추출하는 부분 #######################################

PATH = '/content/drive/MyDrive/기계학습 1등팀/data/baduk_net5.pth'
net = torch.load(PATH)
net.eval()

print(len(testloader))
for i, data in enumerate(testloader,0):
        inputs, labels = data  # data입력
        outputs = net(inputs)
        print(i)

np.save('/content/drive/MyDrive/기계학습1등팀/data/test_X.npy', X)